In [ ]:
%config Completer.use_jedi=False
import numpy as np
import pandas as pd
import scipy.stats as sps
import math
from scipy.special import gamma
import matplotlib.pyplot as plt

In [ ]:
def BlackScholesCallPut(S, K, T, sigma, r, call_put=1):
    d1 = (np.log(S/K) + (r+.5*sigma**2)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return call_put*(S*norm.cdf(call_put*d1) - K*np.exp (-r*T) * norm.cdf (call_put*d2))

def impliedVol(S, K, T, r, price):
    def smileMin(vol, *args):
        S, K, T, r, price = args
        return price - BlackScholesCallPut(S, K, T, vol, r, 1)
    vMin = 0.0001
    vMax = 3.
    return bisect(smileMin, vMin, vMax, args=(S, K, T, r, price), rtol=1e-15, full_output=False, disp=True)

### Monte Carlo simulation
#### 1. Heston model
$$
\begin{aligned}
    dS_t &= \sqrt{V_t}S_tdW_t, \quad \text{for}\; t>0,\,S_0=s_0,\\
    dV_t &= \kappa(\theta-V_t)dt+\sigma\sqrt{V_t}dB_t,\quad \text{for}\; t>0,\,V_0=v_0,
\end{aligned}
$$
with $W,\,B$ brownian motions of the correlation $dW_tdZ_t=\rho\in[-1,\,1],\; \kappa,\,\theta,\,\sigma,\,v_0>0$ and $2\kappa\theta>\sigma^2$. 

The parameters to calibrate are $(\kappa,\,\rho,\,V0)\in\mathbb{R}^4$


#### 2. Rough Heston model
$$
\begin{aligned}
dS_t &= S_t\sqrt{V_t}dW_t\\
V_t &= V_0+\frac{\lambda}{\Gamma(\alpha)}\int_0^t(t-s)^{\alpha-1}(\theta-V_s)ds+\frac{\lambda v}{\Gamma(\alpha)}\int_0^t(t-s)^{\alpha-1}\sqrt{V_s}dB_s
\end{aligned}
$$
with $\langle dW_t, \, dB_t\rangle=\rho dt$ and $\alpha\in(1/2, 1)$.

The parameters to calibrate are $v=(\alpha, \lambda, v, v0)\in\mathbb{R}^4$

#### 3. Quadratic rough Heston model
$$
\begin{aligned}
dS_t&=S_t\sqrt{V_t}dW_t \\
V_t&=a(Z_t-b)^2+c
\end{aligned}
$$
where $W$ is a Brownian motion and $a,\,b,\,c>0$. This model is of rought Heston type, in the sense that weighted past price returns are drivers of the volatility dynamics:
$$Z_t=\int_0^t(t-s)^{\alpha-1}\frac{\lambda}{\Gamma(\alpha)}(\theta_0(s)-Z_s)ds+\int_0^t(t-s)^{\alpha-1}\frac{\lambda}{\Gamma(\alpha)}\eta\sqrt{V_s}dW_s$$
with $\alpha\in(1/2,\,1),\,\lambda>0,\,\eta>0$ and $\theta_0$ a deterministic function.



For simplicity, $\theta_0(t)=\cfrac{Z_0}{\lambda\Gamma(1-\alpha)}t^{-\alpha}$, so we have
$$Z_t=Z_0-\int_0^t(t-s)^{\alpha-1}\frac{\lambda}{\Gamma(\alpha)}Z_sds+\int_0^t(t-s)^{\alpha-1}\frac{\lambda}{\Gamma(\alpha)}\eta\sqrt{V_s}dW_s$$

The parameters to calibrate are $v=(\alpha, \lambda, a,b,c, Z_0)\in\mathbb{R}^6$(Note that we can always take $\eta=1$ up to a rescaling of the other parameters).

In [ ]:
class MC:
    """
    A Monte Carlo class for simulating the stochastic models (ex: Heston, rough Heston...)
    
    Output: Prices: a list of prices
            Spot Variances
     
    """
    def __init__(self, s0=100, T=28, dt=0.01):
        self.s0 = s0 ### Price at t=0
        self.T  = T  ### Time to maturity
        self.dt = dt ### time interval for the Monte Carlo
        
    
        
    def HestonMC(self, k, theta, sigma, v0, rho):
        """
        Monte Carlo Simulation for the Heston model
        
        """
        
        # Generate a 2-dimensional Brownian Motion sequence
        Mu  = np.zeros(2)
        Cov = np.array([[1, rho], [rho, 1]])
        N   = int(self.T/self.dt)
        W   = np.random.multivariate_normal(Mu, Cov, N)
        W_t = W[:, 0]
        B_t = W[:, 1]

        # Generate paths
        Vt    = np.zeros(N)
        Vt[0] = v0

        Xt    = np.zeros(N)
        Xt[0] = np.log(self.s0)

        for i in range(1, N):
            tmp = np.max(Vt[i-1], 0)
            Vt[i] = Vt[i-1] + k*(theta- tmp)*dt + sigma*np.sqrt(tmp*dt)*B_t[i]
            Xt[i]  = Xt[i-1] -0.5*tmp*dt + np.sqrt(tmp*dt)*W_t[i]
        
        return np.exp(Xt), Vt
    
    def rHestonMC(self, alpha, Lambda, rtheta, v, v0):
        """
        Monte Carlo Simulation for the rough Heston model
        """
        
        # Generate a 2-dimensional Brownian Motion sequence
        Mu  = np.zeros(2)
        Cov = np.array([[1, rho], [rho, 1]])
        N   = int(self.T/self.dt)
        W   = np.random.multivariate_normal(Mu, Cov, N)
        W_t = W[:, 0]
        B_t = W[:, 1]

        # Initialize S, V, Z
        Vt    = np.zeros(N)
        Xt    = np.zeros(N)
        Xt[0] = np.log(self.s0)
        Vt[0] = v0

        # Set the step for the inner Monte Carlo
        t = np.linspace(0, T, N)
        coef = Lambda/gamma(alpha)

        for i in range(1, N):
            # Genrate Z_ti by Monte Carlo
            ti   = np.power(t[i] - t[: i], alpha - 1)
            Vi   = theta - Vt[: i]
            Bi   = B_t[: i]*np.sqrt(dt)
            Vt[i] = v0 - coef*dt*np.sum(ti*Vi) + v*coef*np.sum(ti*np.sqrt(Vi)*Bi)
            Xt[i] = Xt[i-1] -0.5*Vt[i-1]*dt + np.sqrt(Vt[i-1]*dt)*W_t[i]

        return np.exp(Xt), Vt
    
    def qrHestonMC(self, alpha, qLambda, Z0, a, b, c):
        """
        Monte Carlo Simulation for the quadratic rough Heston model
        """
        
        # Generate a Brownian Motion sequence
        N = int(self.T/self.dt)
        W = np.random.normal(0, 1, N)

        # Initialize S, V, Z
        Vt    = np.zeros(N)
        Z    = np.zeros(N)
        Xt    = np.zeros(N)
        Xt[0] = np.log(self.s0)
        Z[0] = Z0
        Vt[0] = a*(Z0-b)**2 + c

        # Set the step for the inner Monte Carlo
        t = np.linspace(0, T, N)
        coef = qLambda/gamma(alpha)

        for i in range(1, N):
            # Genrate Z_ti by Monte Carlo
            ti   = np.power(t[i] - t[: i], alpha - 1)
            Zi   = Z[: i]
            Vi   = Vt[: i]
            Wi   = W[: i]*np.sqrt(dt)
            Z[i] = Z0 - coef*dt*np.sum(ti*Zi) + coef*np.sum(ti*np.sqrt(Vi)*Wi)
            Vt[i] = a*(Z[i]-b)**2 + c
            Xt[i] = Xt[i-1] -0.5*Vt[i-1]*dt + np.sqrt(Vt[i-1]*dt)*W[i]

        return np.exp(Xt), Vt
    
    
        

In [ ]:
s0 = 100
T = 0.8
dt = 0.001
alpha = 0.6 ### alpha = H+1/2
#### Parameters for Heston
k = 0.8
theta = 0.5
rho = -0.6
sigma = 0.8
v0 = 0.05

#### Parameters for rough Heston
Lambda = 1.5
rtheta = 0.2
v = 0.3
v0 = 0.1

#### Parameters for quadratic rough Heston
# alpha = np.linspace(0.5, 1.0, 100)
# Lambda = np.linspace(0, 10, 100)
# Z0 = np.linspace(0, 1, 100)
# a = np.linspace(0, 1, 100)
# b = np.linspace(0,1,100)
# c = np.linspace(0, 0.1, 100)
Z0 = 0.1
a = 0.384
b = 0.095
c = 0.0025
qLambda = 1.2


In [ ]:
mc = MC(s0, T, dt)
rS, rV = mc.qrHestonMC(alpha, qLambda, Z0, a, b, c)
# rS, rV = mc.qrHestonMC(alpha, Lambda, rtheta, v, v0)
time = np.linspace(0, T, int(T/dt))
f = plt.figure(figsize=(16,4))
ax1 = plt.subplot(121)
ax1.plot(time, rS, label = "qrHeston")
ax1.grid()
ax2 = plt.subplot(122)
ax2.plot(time, np.sqrt(rV), label = "qrHeston")
ax2.grid()

In [ ]:
mc = MC(s0, T, dt)
S, V = mc.HestonMC(k, theta, sigma, v0, rho)
rS, rV = mc.rHestonMC(alpha, Lambda, rtheta, v, v0)
qrS, qrV = mc.qrHestonMC(alpha, qLambda, Z0, a, b, c)
# vol = np.sqrt
time = np.linspace(0,T, int(T/dt))
f = plt.figure(figsize=(18,6))
ax1 = plt.subplot(121)
ax1.plot(time, S, label = "Heston")
ax1.plot(time, rS, label = "rHeston")
ax1.plot(time, qrS, label = "qrHeston")
ax1.grid()
ax1.set_xlabel("time")
ax1.set_ylabel("Prices")
ax1.legend(loc=1)
ax2 = plt.subplot(122)
ax2.plot(time, np.sqrt(V), label="Heston")
ax2.plot(time, np.sqrt(rV), label = "rHeston")
ax2.plot(time, np.sqrt(qrV), label = "qrHeston")

ax2.grid()
ax2.set_xlabel("time")
ax2.set_ylabel("instaneous volatility")
# plt.title("11")
ax2.legend(loc=1)
plt.show()

In [ ]:
def price(T=30, S0=100, r = 0, N = 1000, label='call', model = 'HestonMC', para = None):
    """
    Calculate the call/put price for given Strike K and maturity T
    Input:
        T:      float, time to Maturity
        K:      float, Strike price
        r:      float, risk-free rate
        label:  string, "call/put" for Call/Put European options
        model:  string, model used for pricing
        para:   a list or an array, parameters of the model
        
    Output:
        Price of the Call/Put European option 
    
    """
    payoff = 0
    mod = getattr(MC, model)
    for n in range(N):
        s, _=  HestonMC(para)[-1]
        payoff += np.max(s - K, 0)
    payoff /= N
    
    return payoff*np.exp(-r*T)
  
    

In [ ]:
Lambda = 1.6
rtheta = 0.2
v = 0.1
v0 = 0.4
mc = MC(s0, T, dt)
rS, rV = mc.rHestonMC(alpha, Lambda, rtheta, v, v0)
time = np.linspace(0, T, int(T/dt))
f = plt.figure(figsize=(18,6))
ax1 = plt.subplot(121)
ax1.plot(time, rS, label = "rHeston")
ax1.grid()
x2 = plt.subplot(122)
ax2.plot(time, np.sqrt(rV), label = "rHeston")
ax2.grid()


In [ ]:
rV

In [ ]:
#### Milstein method


In [ ]:
num_sim = 1
t_init = 0
t_end = 1
N = 1000
dt = float(t_end-t_init)/N
y_init = 1
mu = 3
sigma = 1

In [ ]:
ts = np.arange(t_init, t_end, dt)
ys = np.zeros(N)
ys[0] = y_init

for _ in range(num_sim):
    for i in range(1, ts.size):
        t = (i-1)*dt
        y = ys[i-1]
        dWt = np.random.normal(loc= 0.0, scale=np.sqrt(dt))
        ys[i] = y+mu*dt*y + sigma*y*dWt + 0.5*sigma**2 * (dWt**2 - dt)
    plt.plot(ts, ys)
plt.xlabel("time(s)")
plt.grid()
h = plt.ylabel("y")
h.set_rotation(0)
plt.show()

In [ ]:
dir(cmath)

In [ ]:
cmath.phase(1.2)

In [ ]:
rng = np.random.default_rng(123)
rng.gamma(1)
rng.random()

In [ ]:
from numpy.random import Generator
?Generator.integers

In [2]:
import write_read
%run write_read

In [1]:
import multiprocessing
multiprocessing.cpu_count()

4

In [4]:
from multiprocessing import Process, Queue
import os, time, random

# 写数据进程执行的代码:
def write(q):
    print('Process to write: {}'.format(os.getpid()))
    for value in ['A', 'B', 'C']:
        print('Put %s to queue...' % value)
        q.put(value)
        time.sleep(random.random())

# 读数据进程执行的代码:
def read(q):
    print('Process to read:{}'.format(os.getpid()))
    while True:
        value = q.get(True)
        print('Get %s from queue.' % value)

# if __name__=='__main__':
# 父进程创建Queue，并传给各个子进程：
q = Queue()
pw = Process(target=write, args=(q,))
pr = Process(target=read, args=(q,))
# 启动子进程pw，写入:
pw.start()
# 启动子进程pr，读取:
pr.start()
# 等待pw结束:
pw.join()
# pr进程里是死循环，无法等待其结束，只能强行终止:
pr.terminate()